In [1]:
from zipfile import ZipFile

In [2]:
with ZipFile('parkinsons.zip','r') as zipObj:
  zipObj.extractall()

#Image preprocessing of spiral drawings

In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile as zf
import os
import random
import cv2
import imutils
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from skimage import feature
from keras.utils import to_categorical
from imutils import build_montages,paths
from google.colab.patches import cv2_imshow
sns.set()

In [4]:
os.getcwd()

'/content'

In [11]:
spiral_train_healthy = os.listdir('/content/parkinsons/spiral/training/healthy/')
spiral_train_park = os.listdir('/content/parkinsons/spiral/training/parkinson/')

fp_spiral_train_healthy = '/content/parkinsons/spiral/training/healthy/'
fp_spiral_train_park = '/content/parkinsons/spiral/training/parkinson/'

spiral_test_healthy = os.listdir('/content/parkinsons/spiral/testing/healthy/')
spiral_test_park = os.listdir('/content/parkinsons/spiral/testing/parkinson/')

fp_spiral_test_healthy = '/content/parkinsons/spiral/testing/healthy/'
fp_spiral_test_park = '/content/parkinsons/spiral/testing/parkinson/'

In [12]:
def quantify_image(image):
  features = feature.hog(image,orientations=9,
                pixels_per_cell=(10,10),cells_per_block=(2,2),transform_sqrt=True,block_norm="L1")

  return features

In [13]:
trainX = []
testX = []
outputs = []
trainY = []
testY = []

for i in spiral_train_healthy:
  image = cv2.imread(fp_spiral_train_healthy+i)
  image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image , (200,200))
  image =cv2.threshold(image, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  features = quantify_image(image)
  trainX.append(features)
  trainY.append('healthy')

for i in spiral_train_park:
  image = cv2.imread(fp_spiral_train_park+i)
  image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image , (200,200))
  image = cv2.threshold(image ,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  features = quantify_image(image)
  trainX.append(features)
  trainY.append('parkinson')

for i in spiral_test_healthy:
  image = cv2.imread(fp_spiral_test_healthy+i)
  outputs.append(image)
  image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image , (200,200))
  image = cv2.threshold(image ,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  features = quantify_image(image)
  testX.append(features)
  testY.append('healthy')

for i in spiral_test_park:
  image = cv2.imread(fp_spiral_test_park+i)
  outputs.append(image)
  image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image , (200,200))
  image = cv2.threshold(image ,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  features = quantify_image(image)
  testX.append(features)
  testY.append('parkinson')

In [14]:
trainX = np.array(trainX)
testX = np.array(testX)
trainY = np.array(trainY)
testY = np.array(testY)

In [15]:
trainX

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
testX

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
trainY

array(['healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy',
       'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy',
       'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy',
       'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy',
       'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy',
       'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy',
       'parkinson', 'parkinson', 'parkinson', 'parkinson', 'parkinson',
       'parkinson', 'parkinson', 'parkinson', 'parkinson', 'parkinson',
       'parkinson', 'parkinson', 'parkinson', 'parkinson', 'parkinson',
       'parkinson', 'parkinson', 'parkinson', 'parkinson', 'parkinson',
       'parkinson', 'parkinson', 'parkinson', 'parkinson', 'parkinson',
       'parkinson', 'parkinson', 'parkinson', 'parkinson', 'parkinson',
       'parkinson', 'parkinson', 'parkinson', 'parkinson', 'parkinson',
       'parkinson'], dtype='<U9')

In [18]:
testY

array(['healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy',
       'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy',
       'healthy', 'healthy', 'healthy', 'parkinson', 'parkinson',
       'parkinson', 'parkinson', 'parkinson', 'parkinson', 'parkinson',
       'parkinson', 'parkinson', 'parkinson', 'parkinson', 'parkinson',
       'parkinson', 'parkinson', 'parkinson'], dtype='<U9')

In [19]:
le = LabelEncoder()
trainY = le.fit_transform(trainY)
testY = le.transform(testY)

In [20]:
trainY

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [21]:
testY

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])

#Image preprocessing of wave drawings

In [22]:
wave_train_healthy = os.listdir('/content/parkinsons/wave/training/healthy/')
wave_train_park = os.listdir('/content/parkinsons/wave/training/parkinson/')

fp_wave_train_healthy = '/content/parkinsons/wave/training/healthy/'
fp_wave_train_park = '/content/parkinsons/wave/training/parkinson/'

wave_test_healthy = os.listdir('/content/parkinsons/wave/testing/healthy/')
wave_test_park = os.listdir('/content/parkinsons/wave/testing/parkinson/')

fp_wave_test_healthy = '/content/parkinsons/wave/testing/healthy/'
fp_wave_test_park = '/content/parkinsons/wave/testing/parkinson/'

In [29]:
def quantify_image1(image1):
  features1 = feature.hog(image1,orientations=9,
                pixels_per_cell=(10,10),cells_per_block=(2,2),transform_sqrt=True,block_norm="L1")

  return features1

In [30]:
trainX1 = []
testX1 = []
outputs1 = []
trainY1 = []
testY1 = []

for i in wave_train_healthy:
  image1 = cv2.imread(fp_wave_train_healthy+i)
  image1 = cv2.cvtColor(image1 , cv2.COLOR_BGR2GRAY)
  image1 = cv2.resize(image1 , (200,200))
  image1 =cv2.threshold(image1, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  features1 = quantify_image1(image1)
  trainX1.append(features1)
  trainY1.append('healthy')

for i in wave_train_park:
  image1 = cv2.imread(fp_wave_train_park+i)
  image1 = cv2.cvtColor(image1 , cv2.COLOR_BGR2GRAY)
  image1 = cv2.resize(image1 , (200,200))
  image1 = cv2.threshold(image1 ,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  features1 = quantify_image1(image1)
  trainX1.append(features1)
  trainY1.append('parkinson')

for i in wave_test_healthy:
  image1 = cv2.imread(fp_wave_test_healthy+i)
  outputs1.append(image1)
  image1 = cv2.cvtColor(image1 , cv2.COLOR_BGR2GRAY)
  image1 = cv2.resize(image1 , (200,200))
  image1 = cv2.threshold(image1 ,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  features1 = quantify_image1(image1)
  testX1.append(features1)
  testY1.append('healthy')

for i in wave_test_park:
  image1 = cv2.imread(fp_wave_test_park+i)
  outputs1.append(image1)
  image1 = cv2.cvtColor(image1 , cv2.COLOR_BGR2GRAY)
  image1 = cv2.resize(image1 , (200,200))
  image1 = cv2.threshold(image1 ,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  features1 = quantify_image1(image1)
  testX1.append(features1)
  testY1.append('parkinson')

In [31]:
trainX1 = np.array(trainX)
testX1 = np.array(testX)
trainY1 = np.array(trainY)
testY1 = np.array(testY)

In [32]:
trainX1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
testX1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
trainY1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [35]:
testY1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])

In [36]:
le = LabelEncoder()
trainY1 = le.fit_transform(trainY)
testY1 = le.transform(testY)

In [37]:
trainY1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [38]:
testY1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])